# Enverionment

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import warnings

warnings.filterwarnings("ignore")
# change to ur local dir where 'Sampler' located"
base_dir = "/data/hujing"
working_dir = os.path.join(base_dir, "RFantibody")

sys.path.append(os.path.join(working_dir, "src/rfantibody"))

* We've known how the forward process implemented and we need the reverse process --> "denoised process" to push the noisy coordinated to meaningful structures.
* According to RFAntibody code, the authors use the architecture of RoseTTAFold2 to re-construct the 3D-structure. So let's dive into the model framework firstly!

* [Inputs for the models, Page 17](https://www.biorxiv.org/content/10.1101/2024.03.14.585103v1.supplementary-material?versioned=true):
  * MSA (virtual)
  * Masked sequence
  * noised_coordinates
  * residue idx
  * time_step T
  * and some prcessed features

* Let's check how the features are processed.

In [2]:
# Firstly, we need to load some parameters. For sinplicity, we've prepared a yaml file which provides all parameters we need.
# One can change the yaml file for the file path parameters.

from rfantibody.rfdiffusion.inference import model_runners
from rfantibody.rfdiffusion.inference.ab_util import featurize
import hydra
from omegaconf import OmegaConf
import torch

config_path = "/data/hujing/RFantibody/" # change to your own yaml dir on ur PC
config_name = "init_conf"

with hydra.initialize_config_dir(config_dir=config_path):
    conf = hydra.compose(config_name=config_name)

# initialize the sampler for antibody
ckpt_path = "/data/hujing/RFantibody/weights/RFdiffusion_Ab.pt"
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

#ckpt = torch.load(ckpt_path, map_location=device)

sampler = model_runners.AbSampler(conf)

This is inf_conf.ckpt_path
/data/hujing/RFantibody/weights/RFdiffusion_Ab.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][SE3_param_full] = {'num_layers': 1, 'num_channels': 32, 'num_degrees': 2, 'n_heads': 4, 'div': 4, 'l0_in_features': 8, 'l0_out_features': 8, 'l1_in_features': 3, 'l1_out_features': 2, 'num_edge_features': 32}
USING MODEL CONFIG: self._conf[model][SE3_param_topk] = {'num_layers': 1, 'num_channels': 32, 'num_degrees': 2, 'n_heads': 4, 'div': 4, 'l0_in_features': 64, 'l0_out_features': 64, 'l1_in_features': 3, 'l1_out_features': 2, 'num_edge_features': 64}
USING MODEL CONFIG: self._conf[model][d_hidden] = 32
USING MODEL CONFIG: self._conf[model][d_hidden_templ] = 32
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USING MODEL CONFIG: self._conf[model][d_pair] = 128
USING MODEL CONFIG: self._conf[model][d_templ] = 64
USING MODEL CONFIG: self._conf[model

ic| 'Doing AR Sequence Decoding'


 
----------------------------------------------------------------------------------------------------
 
Delete these in inference/model_runners.py once they are implemented/once you decide they are not required for inference -- JW
 -predict_previous
 -prob_self_cond
 -seqdiff_b0
 -seqdiff_bT
 -seqdiff_schedule_type
 -seqdiff
 -freeze_track_motif
 -use_motif_timestep
 
----------------------------------------------------------------------------------------------------
 
Successful diffuser __init__


In [3]:
x_init, seq_init = sampler.sample_init() # this step is what we did in the Sampler section n last notebook

ic| self.pose.length(): 474
ic| self.pose.binder_len(): 223
ic| self.pose.L.seq: array([ 3,  9,  5, 12, 16,  5, 15, 14, 15, 15, 10, 15,  0, 15, 19,  7,  3,
                             1, 19, 16,  9, 16,  4,  1,  0, 15,  5,  3, 19,  2, 16,  0, 19,  0,
                            17, 18,  5,  5, 11, 14,  7, 11,  0, 14, 11, 10, 10,  9, 18, 15,  0,
                            15, 13, 10, 18, 15,  7, 19, 14, 15,  1, 13, 15,  7, 15,  1, 15,  7,
                            16,  3, 13, 16, 10, 16,  9, 15, 15, 10,  5, 14,  6,  3, 13,  0, 16,
                            18, 18,  4,  5,  5,  8, 18, 16, 16, 14, 14, 16, 13,  7,  5,  7, 16,
                            11, 19,  6,  9, 11])
ic| self.pose.length(): 478
ic| self.pose.binder_len(): 227
ic| self.pose.L.seq: array([ 3,  9,  5, 12, 16,  5, 15, 14, 15, 15, 10, 15,  0, 15, 19,  7,  3,
                             1, 19, 16,  9, 16,  4,  1,  0, 15,  5,  3, 19,  2, 16,  0, 19,  0,
                            17, 18,  5,  5, 11, 14,  7, 11,  0,

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547
Using cached chi_beta_T dictionary.
Done calculating chi_beta_T, chi_alphas_T, and chi_abars_T dictionaries.


, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                                    False, False, False, False, False, False, False, False, False, False,
                       

* Assume we satrt from step 50, the denoised to step 0
* The following steps are how it inplemented
* For the descriptons of each input features, plz refer to [Supplementary Methods Table2](https://www.biorxiv.org/content/10.1101/2024.03.14.585103v1.supplementary-material?versioned=true).

In [52]:
from rfantibody.rfdiffusion.inference.ab_util import featurize, process_init_selfcond, process_selfcond
import numpy as np

t = 50
x_t, seq_t = x_init.clone(), seq_init.clone()

L = x_t.shape[0] # get the total length
tmp_xyz = torch.full((L, 27, 3), np.nan)
tmp_xyz[:, :14] = x_t

features = featurize(sampler.ab_item,
                     seq_t,
                     tmp_xyz,
                     sampler.preprocess_conf.d_t1d,
                     sampler.preprocess_conf.hotspot_dim,
                     sampler.ab_conf.T_scheme,
                     1 - (t / sampler.T),
                     ~sampler.preprocess_conf.motif_sidechain_input,
                     ~sampler.ab_conf.no_bugfix_t1d_mask)

idx_pdb = torch.arange(L) 
if sampler.ab_item.target:
    idx_pdb[sampler.ab_item.target_mask] += 200 # Do idx jump at chainbreak

## Add hotspots to t1d ##
#########################
features['t1d'][...,22] = sampler.ab_item.hotspots[None,None]

retval = (
          features['msa_masked'],
          features['msa_full'],
          features['seq'],
          features['xyz_prev'].unsqueeze(0),
          idx_pdb.unsqueeze(0),
          features['t1d'].unsqueeze(0),
          features['t2d'].unsqueeze(0),
          features['xyz_t'].unsqueeze(0),
          features['alpha_t'].unsqueeze(0)
         )

# Send all inputs to device
retval = [i.to(device) for i in retval]

msa_masked, msa_full, seq_in, xt_in, idx_pdb, t1d, t2d, xyz_t, alpha_t = retval
sc2d, xyz_sc = process_init_selfcond(t2d, xyz_t, sampler.ab_conf, xyz_t.device)

msa_prev = None

ic| f'Featurizing with {T_scheme}': 'Featurizing with single_T_correct_selfcond'


In [10]:
sampler.ab_conf['T_scheme']

'single_T_correct_selfcond'

* We've processed the features ready to the model, so let's check the architecture of the RosseTTAFold now
* The RosseTTAFold is kinda straidforward, the model structure is as follows:
  ![RF workflow](RFM.jpg)
  

* You can call the model as follows:

In [14]:

from rfantibody.rfdiffusion.RoseTTAFoldModel import RoseTTAFoldModule

RosettaModel = RoseTTAFoldModule(**sampler._conf.model, 
                                 d_t1d=sampler.d_t1d, 
                                 d_t2d=sampler.d_t2d, 
                                 use_selfcond_emb=sampler.use_selfcond_emb, 
                                 T=sampler._conf.diffuser.T).to(device)

RosettaModel.eval()

test = RosettaModel(msa_masked[...,:10, :],
                    msa_full[..., :10, :],
                    seq_in[:, :10, :],
                    xt_in[:, :10, ...],
                    idx_pdb[:, :10],
                    t1d=t1d[..., :10, :],
                    t2d=t2d[..., :10, :10, :],
                    sc2d=sc2d,
                    xyz_sc=xyz_sc,
                    xyz_t=xyz_t[..., :10, :, :],
                    alpha_t=alpha_t[..., :10, :],
                    msa_prev = msa_prev,
                    pair_prev = None,
                    state_prev = None,
                    t=torch.tensor(t),
                    return_infer=True,
                    motif_mask=sampler.diffusion_mask[:10].squeeze().to(device))

In [16]:
test[0].shape

torch.Size([1, 10, 256])

* Next, we'll go through each sub-module of the RosettaFold.
# MSA embedding
* We don't really need mutiple-sequence alignment here since we only have one masked sequence. I guess the authors wanted to align to the original RF model.
* For the following sub-module, I'll show you the detailed inplementation code.

In [18]:
from rfantibody.rfdiffusion.Embeddings import MSA_emb

d_msa = sampler._conf.model.d_msa
d_pair = sampler._conf.model.d_pair
d_state = 32
p_drop = sampler._conf.model.p_drop
input_seq_onehot = sampler._conf.model.input_seq_onehot

latent_emb = MSA_emb(d_msa=d_msa, d_pair=d_pair, d_state=d_state,
                p_drop=p_drop, input_seq_onehot=False).to(device)

msa_latent, pair, state = latent_emb(msa_masked[...,:10, :],
                                     seq_in[:, :10, :],
                                     idx_pdb[:, :10])

In [19]:
msa_latent.shape, pair.shape, state.shape

(torch.Size([1, 1, 10, 256]),
 torch.Size([1, 10, 10, 128]),
 torch.Size([1, 10, 32]))

* As you can see above, the **latent_emb** module output three variables, the latent msa emb, pair emb and state repr.
* In the original code, the author used a **one-hot trick** by Sergey. I kepting using the official API for simplicity.
* The authors also provided some custome initialization methods, I also replace it with official API.
* Next is how it implemented

In [44]:
import torch.nn as nn
import torch.nn.init as init
from rfantibody.rfdiffusion.Embeddings import PositionalEncoding2D

class MSA_emb_revised(nn.Module):
    def __init__(self, d_msa=256, 
                         d_pair=128,
                        d_state=32,
                        p_drop=0.1,
                        input_seq_onehot=False,
                        d_init=48,
                        minpos=-32,
                        maxpos=32):
        super().__init__()
        
        self.emb = nn.Linear(d_init, d_msa)
        self.seq_emb1 = nn.Linear(22, d_msa, bias=False)
        init.kaiming_normal_(self.seq_emb1.weight, mode='fan_in', nonlinearity='linear')

        self.drop = nn.Dropout(p_drop)

        self.emb_left = nn.Linear(22, d_pair, bias=False)
        self.emb_right = nn.Linear(22, d_pair, bias=False)
        init.kaiming_normal_(self.emb_left.weight, mode='fan_in', nonlinearity='linear')
        init.kaiming_normal_(self.emb_right.weight, mode='fan_in', nonlinearity='linear')
        self.pos = PositionalEncoding2D(d_pair, minpos=minpos, maxpos=maxpos, p_drop=p_drop)

        self.emb_state = nn.Linear(22, d_state, bias=False)
        init.kaiming_normal_(self.emb_state.weight, mode='fan_in', nonlinearity='linear')
        
    def forward(self, msa, seq, idx):
        """
        msa: [B, N, L, d_init]
        seq: [B, L, 22]
        idx: [B, L]
        """
        N = msa.shape[1] # N represents the number of seqs in MSAs, here's always 1!
        msa_latent = self.emb(msa) # [B, N, L, d_msa]
        
        tmp = self.seq_emb1(seq).unsqueeze(1) # [B, 1, L, d_msa]
        msa_latent += tmp.expand(-1, N, -1, -1)
        msa_latent = self.drop(msa_latent)
        
        left = self.emb_left(seq).unsqueeze(2)
        right = self.emb_right(seq).unsqueeze(1)

        pair = left + right
        #print(left.shape, right.shape, pair.shape)
        pair += self.pos(pair, idx)

        state = self.drop(self.emb_state(seq))

        return msa_latent, pair, state

MSA_emb_revised
latent_emb_revised = MSA_emb_revised(d_msa=d_msa, d_pair=d_pair, d_state=d_state,
                p_drop=p_drop, input_seq_onehot=False).to(device)

msa_latent_rv, pair_rv, state_rv = latent_emb_revised(msa_masked[...,:10, :],
                                     seq_in[:, :10, :],
                                     idx_pdb[:, :10])

In [45]:
msa_latent_rv.shape, pair_rv.shape, state_rv.shape

(torch.Size([1, 1, 10, 256]),
 torch.Size([1, 10, 10, 128]),
 torch.Size([1, 10, 32]))

* As shown above, the output tensors have identical shapes. However, their actual values differ due to the distinct weight initialization methods we employed.

* The denoised model retains the extra_msa module—referred to here as **full_emb**. Let’s now examine what this extra_msa (i.e., full_emb) module actually does.

In [53]:
from rfantibody.rfdiffusion.Embeddings import Extra_emb

d_msa_full = sampler._conf.model.d_msa_full

full_emb = Extra_emb(d_msa=d_msa_full, d_init=25,
                p_drop=p_drop, input_seq_onehot=False).to(device)

msa_full_ot = full_emb(msa_full[...,:10, :],
                     seq_in[:, :10, :],
                     idx_pdb[:, :10])

In [58]:
msa_full_ot.shape

torch.Size([1, 1, 10, 64])

* We’ll now implement Extra_emb step by step, just as we did before.

In [57]:
class Extra_emb_revised(nn.Module):
    def __init__(self, d_msa=256,
                        d_init=25,
                        p_drop=0.1,
                        input_seq_onehot=False):
        super().__init__()
        self.emb = nn.Linear(d_init, d_msa)
        self.emb_q = nn.Linear(22, d_msa, bias=False)
        init.kaiming_normal_(self.emb_q.weight,  mode='fan_in', nonlinearity='linear')
        self.drop = nn.Dropout(p_drop)
        
    def forward(self, msa, seq, idx):
        """
        msa: [B, N, L, d_init]
        seq: [B, L, 22]
        idx: [B, L]
        """
        N = msa.shape[1]
        full_msa = self.emb(msa) # [B, N, L, d_msa]
        seq = self.emb_q(seq).unsqueeze(1) # [B, 1, L, d_msa]

        full_msa += seq.expand(-1, N, -1, -1) 

        return self.drop(full_msa)

full_emb_rv = Extra_emb_revised(d_msa=d_msa_full, d_init=25,
                p_drop=p_drop, input_seq_onehot=False).to(device)

msa_full_ot_rv = full_emb(msa_full[...,:10, :],
                     seq_in[:, :10, :],
                     idx_pdb[:, :10])

msa_full_ot_rv.shape

torch.Size([1, 1, 10, 64])

# RECYCLE Module
* This module accepts previous iterations as input, information is passed through network iterations.
* The initial **msa_prev**, **pair_prev** and **state_prev** can be set to None.

In [72]:
from rfantibody.rfdiffusion.Embeddings import Recycling
import torch

msa_prev = torch.zeros_like(msa_latent[:, 0])
pair_prev = torch.zeros_like(pair)
state_prev = torch.zeros_like(state)

recycle = Recycling(d_msa=d_msa, d_pair=d_pair, d_state=d_state).to(device)
msa_recycle, pair_recycle, state_recycle = recycle(seq_in[:, :10, :],
                                                   msa_prev, 
                                                   pair_prev, 
                                                   xt_in[:, :10, ...], 
                                                   state_prev)

In [87]:
from rfantibody.rfdiffusion.util_module import rbf

class Recycling_revised(nn.Module):
    def __init__(self,d_msa=256, d_pair=128, d_state=32):
        super().__init__()
        self.proj_dist = nn.Linear(36+d_state*2, d_pair)
        self.norm_state = nn.LayerNorm(d_state)
        self.norm_pair = nn.LayerNorm(d_pair)
        self.norm_msa = nn.LayerNorm(d_msa)

        # reset parameters here
    def forward(self, seq, msa, pair, xyz, state):
        """
        seq: [B, L, 22]
        msa: [B, N, L, d_init]
        pair: [B, L, L, 128]
        xyz: [B, L, 27, 3]
        state: [B, L, 32]
        """
        B, L = seq.shape[:2]
        msa = self.norm_msa(msa)
        state = self.norm_state(state)

        left = state.unsqueeze(2).expand(-1, -1, L, -1) # [B, L, L, 64]
        right = state.unsqueeze(1).expand(-1, L, -1, -1)

        Nitro = xyz[:, :, 0, :] # [B, L, 3]
        CA = xyz[:, :, 1, :]
        C = xyz[:, :, 2, :]

        b = CA - Nitro
        c = C -CA
        a = torch.cross(b, c, dim=-1) # orthogonal vector
        CB = -0.58273431*a + 0.56802827*b - 0.54067466*c + CA

        dist = rbf(torch.cdist(CB, CB)) # [B, L, L, 36]
        #print(dist.shape, left.shape, right.shape)
        dist = torch.cat([dist, left, right], dim=-1) # [B, L, L, 36+64*2]
        dist = self.proj_dist(dist)

        pair = dist + self.norm_pair(pair)

        return msa, pair, dist

recycle_rv = Recycling_revised(d_msa=d_msa, d_pair=d_pair, d_state=d_state).to(device)
msa_recycle2, pair_recycle2, state_recycle2 = recycle_rv(seq_in[:, :10, :],
                                                   msa_prev, 
                                                   pair_prev, 
                                                   xt_in[:, :10, ...], 
                                                   state_prev)

* The authors used a residue-style logic to update msa, pair and state

In [91]:
#msa_latent.shape, msa_recycle.shape
msa_latent[:, 0, ...] = msa_latent[:, 0, ...] + msa_recycle
pair += pair_recycle
state += state_recycle

# Timestep Embedding
* timestep embedding is disabled in the example, but you can take a look how it works here.

In [109]:
from rfantibody.rfdiffusion.Embeddings import Timestep_emb

# set l=10 
timestep_embedder = Timestep_emb(input_size=8, 
                              output_size=sampler._conf.model.d_time_emb_proj,
                              T=50,
                              use_motif_timestep=sampler._conf.model.use_motif_timestep).to(device)

time_emb = timestep_embedder(10,
                             torch.tensor(t),
                             sampler.diffusion_mask[:10].squeeze().to(device))

# Template Embedding


In [130]:
from rfantibody.rfdiffusion.Embeddings import Templ_emb

d_templ = sampler._conf.model.d_templ
n_head_templ = sampler._conf.model.n_head_templ
d_hidden_templ = sampler._conf.model.d_hidden_templ
d_t1d, d_t2d = sampler.d_t1d, sampler.d_t2d

templ_emb = Templ_emb(d_pair=d_pair, d_templ=d_templ, d_state=d_state,
                                   n_head=n_head_templ,
                                   d_hidden=d_hidden_templ, p_drop=0.25, d_t1d=d_t1d, d_t2d=d_t2d).to(device)

pair_wt, state_wt = templ_emb(t1d[..., :10, :],
                                     t2d[..., :10, :10, :],
                                     alpha_t[..., :10, :], 
                                     xyz_t[..., :10, :, :],
                                     pair, 
                                     state, 
                                     use_checkpoint=False)



* The template module consists of several key sub-modules, including **PairStr2Pair** and **Attention** modules, the workflow proceeds as ![follows](./templ.png):
* I'll go through the two key components step by step
  * In the original code, the authors named it as **templ_stack**, which is **TemplatePairStack** consists of multiple Pair2StrPair layers. So lets take a look what a single Pair2StrPair performs:
        * This layers consists of two ** BiasedAxialAttention** layer but focus on differnent dim called row_attn and col_attn.

In [150]:
from rfantibody.rfdiffusion.util_module import rbf

test_t1d, test_t2d, test_alphat, test_coords, test_pair, test_state = t1d[..., :10, :], \
                                                                     t2d[..., :10, :10, :],\
                                                                     alpha_t[..., :10, :], \
                                                                     xyz_t[..., :10, :, :],\
                                                                     pair, \
                                                                     state

B, T, L, _ = test_t1d.shape

# Prepare 2D template features
left = test_t1d.unsqueeze(3).expand(-1,-1,-1,L,-1)
right = test_t1d.unsqueeze(2).expand(-1,-1,L,-1,-1)
#
templ = torch.cat((test_t2d, left, right), -1) # (B, T, L, L, 90)

emb = nn.Linear(d_t1d*2+d_t2d, d_templ).to(device)
templ = emb(templ).reshape(B*T, L, L, -1) # Template templures (B, T, L, L, d_templ)
# process each template features
test_coords = test_coords.reshape(B*T, L, -1, 3)
rbf_feat = rbf(torch.cdist(test_coords[:,:,1], test_coords[:,:,1]))

* Now we already have the inputs for PairStr2Pair module. Let's see how AxialaAttention work here
* I'll show the row_attention next:
  $out = softmax(Q*K^T/sqrt(L) + bias) * V$

In [166]:
import math

class BiasedAxialAttention_mannual(nn.Module):
    def __init__(self, d_pair, d_bias, n_head, d_hidden, p_drop=0.1, is_row=True):
        super().__init__()

        self.is_row = is_row
        self.norm_pair = nn.LayerNorm(d_pair)
        self.norm_bias = nn.LayerNorm(d_bias)
        
        self.Q = nn.Linear(d_pair, n_head*d_hidden, bias=False)
        self.K = nn.Linear(d_pair, n_head*d_hidden, bias=False)
        self.V = nn.Linear(d_pair, n_head*d_hidden, bias=False)

        self.b = nn.Linear(d_bias, n_head, bias=False)
        self.to_g = nn.Linear(d_pair, n_head*d_hidden)
        self.sigmoid = nn.Sigmoid()

        self.to_out = nn.Linear(n_head*d_hidden, d_pair)
        
        self.n_head, self.d_hidden = n_head, d_hidden
        
        # self.reser_parameter()
        # skip parameter initialization here
        # it should be initialized properly
    def reset_parameter(self):
        nn.init.xavier_uniform_()
        
    def forward(self, templ, rbf_feat):
        """
        templ: [B, L, L, d_pair] as qkv
        rbf: [B, L, L, d_rbf]

        Q,K,V come from templ
        using rbf feat as bias to update
        """
        B, L = templ.shape[:2]

        if self.is_row:
            templ = templ.permute(0,2,1,3)
            rbf_feat = rbf_feat.permute(0,2,1,3)

        Q = self.Q(templ).reshape(B, L, L, self.n_head, self.d_hidden) # [B, L, L, h, d]
        K = self.K(templ).reshape(B, L, L, self.n_head, self.d_hidden)
        V = self.V(templ).reshape(B, L, L, self.n_head, self.d_hidden)

        bias = self.b(rbf_feat) # [B, L, L, h]
        gate = self.sigmoid(self.to_g(templ)) # [B, L, L, h*d]
        
        attn = torch.einsum("bnihd,bnjhd->bijh", Q, K) / math.sqrt(L) # [B, L, L, h]
        attn += bias
        attn = nn.functional.softmax(attn, dim=-1)

        out = torch.einsum("bijh,bkjhd->bikhd", attn, V).reshape(B, L, L, self.n_head * self.d_hidden) # [B, L, L, h*d]
        out *= gate
        
        out = self.to_out(out) # [B, L, L, d_pair)

        if self.is_row:
            out = out.permute(0,2,1,3)

        return out

row_attn = BiasedAxialAttention_mannual(64, 36, n_head=4, d_hidden=32, p_drop=p_drop, is_row=True).to(device)
out = row_attn(templ, rbf_feat)

* Unlike standard attention, which computes interactions between all pairs of tokens, axial attention operates separately along rows or columns.
Moreover, instead of deriving the bias from pairwise features as in AlphaFold 3 (AF3), the authors introduce a distinct bias term.
* Row-wise attention indicates which other residues residue i attends to (i.e., which residues are relevant to i).
* Column-wise attention captures which other residues influence residue j (i.e., which residues provide context for j).
* As evidenced by the operation `torch.einsum("bnihd,bnjhd->bijh", Q, K)`, the attention is computed between the i-th and j-th columns, making the mechanism inherently column-wise.

* Next is the most important module -- **IterativeSimulator**, which consists of several iter blocks
* To ensure dimensional consistency, I applied the following dimensionality transformations:

In [229]:
from rfantibody.rfdiffusion.Track_module import IterativeSimulator

# initilizate the parameters
n_extra_block = sampler._conf.model.n_extra_block
n_main_block = sampler._conf.model.n_main_block
n_ref_block = sampler._conf.model.n_ref_block
n_head_msa = sampler._conf.model.n_head_msa
n_head_pair = sampler._conf.model.n_head_pair
d_hidden = sampler._conf.model.d_hidden
SE3_param_full = sampler._conf.model.SE3_param_full
SE3_param_topk = sampler._conf.model.SE3_param_topk

motif_mask=sampler.diffusion_mask[:10].squeeze().to(device)

simulator = IterativeSimulator(n_extra_block=n_extra_block,
                                            n_main_block=n_main_block,
                                            n_ref_block=n_ref_block,
                                            d_msa=d_msa, d_msa_full=d_msa_full,
                                            d_pair=d_pair, d_hidden=d_hidden,
                                            n_head_msa=n_head_msa,
                                            n_head_pair=n_head_pair,
                                            SE3_param_full=SE3_param_full,
                                            SE3_param_topk=SE3_param_topk,
                                            p_drop=p_drop).to(device)



msa_update, pair_update, R, T, alpha_s, state_update = simulator(seq_in[:, :10, :], 
                                                                 msa_latent, 
                                                                 msa_full_ot, 
                                                                 pair, 
                                                                 xt_in[:, :10, :3],
                                                                 state.repeat(1, 1, 2), 
                                                                 idx_pdb[:, :10], 
                                                                 use_checkpoint=False,
                                                                 motif_mask=motif_mask)

* The *IterativeSimulator* consists of 3 sub-modules: extra_block, main_block and ref_block. The first two sub-modules are based on **IterBlock**.
* Each IterBlock includs 4 types of modules, which are **MSAPairStr2MSA**, **MSA2Pair**, **PairStr2Pair** and **Str2Str**, we'll go through the details of theses modules step by step.

In [253]:
# MSAPaieStr2MSA
from rfantibody.rfdiffusion.Track_module import MSAPairStr2MSA

msa2msa = MSAPairStr2MSA(d_msa=d_msa_full, 
                         d_pair=d_pair,
                          n_head=n_head_msa,
                          d_state=64, #SE3_param['l0_out_features'],
                          use_global_attn=True,
                          d_hidden=32, 
                          p_drop=p_drop).to(device)

xyz = xt_in[:, :10, :3]
rbf_feat = rbf(torch.cdist(xyz[:,:,1,:], xyz[:,:,1,:])) 
msa_ = msa2msa(msa_full_ot, 
        pair, 
        rbf_feat, 
        state_update) # [1, 1, 10, 64]

In [285]:
from rfantibody.rfdiffusion.Attention_module import SequenceWeight

class MSARowAttentionWithBias_mannual(nn.Module):
    def __init__(self, d_msa=64,
                        d_pair=128,
                        n_head=8, 
                        d_hidden=32):
        super().__init__()

        self.norm_msa = nn.LayerNorm(d_msa)
        self.norm_pair = nn.LayerNorm(d_pair)

        self.seq_weight = SequenceWeight(d_msa, n_head, d_hidden, p_drop=0.1)
        self.Q = nn.Linear(d_msa, n_head*d_hidden, bias=False)
        self.K = nn.Linear(d_msa, n_head*d_hidden, bias=False)
        self.V = nn.Linear(d_msa, n_head*d_hidden, bias=False)

        self.B = nn.Linear(d_pair, n_head, bias=False)
        self.G = nn.Linear(d_msa, n_head*d_hidden)
        self.out = nn.Linear(n_head*d_hidden, d_msa)

        self.h, self.dim = n_head, d_hidden

        #reset parameter
        
    def forward(self, msa, pair):
        """
        - msa: MSA feature (B, N, L, d_msa)
        - pair: Pair feature (B, L, L, d_pair)
        """
        B, N, L = msa.shape[:3]
        msa = self.norm_msa(msa)
        pair = self.norm_pair(pair)

        seq_weight = self.seq_weight(msa) # [B, N, L, h, 1]
        
        q = self.Q(msa).reshape(B, N, L, self.h, self.dim) # [B, N, L, h, d]
        k = self.K(msa).reshape(B, N, L, self.h, self.dim)
        v = self.V(msa).reshape(B, N, L, self.h, self.dim)

        bias = self.B(pair) # [B, L, L, h]
        gate = torch.sigmoid(self.G(msa))
        q *= seq_weight.expand(-1, -1, -1, -1, self.dim)
        attn = 1/math.sqrt(self.dim) * torch.einsum("bnqhd,bnkhd->bqkh", q, k)#.reshape(B, L, L, -1)
        attn = nn.functional.softmax(attn + bias, dim=-2) # (B, L, L, h)
        out = torch.einsum("bqkh, bnkhd->bnqhd", attn, v).reshape(B, N, L, -1)
        out *= gate
        out = self.out(out)

        return out
        

class MSAColGlobalAttention_mannual(nn.Module):
    def __init__(self,d_msa=64, 
                        n_head=8, 
                        d_hidden=8):
        super().__init__()
        self.Q = nn.Linear(d_msa, n_head*d_hidden, bias=False)
        self.K = nn.Linear(d_msa, d_hidden, bias=False)
        self.V = nn.Linear(d_msa, d_hidden, bias=False)

        self.B = nn.Linear(d_pair, n_head, bias=False)
        self.G = nn.Linear(d_msa, n_head*d_hidden)
        self.out = nn.Linear(n_head*d_hidden, d_msa)

        self.h, self.dim = n_head, d_hidden
        #reset parameters
        
    def forward(self, msa):
        # msa: [B, N, L, d_msa]
        B, N, L = msa.shape[:3]
        q = self.Q(msa).reshape(B, N, L, self.h, -1) # [B, N, L, h, d]
        q = q.mean(dim=1) # [B, L, h, d]
        k = self.K(msa).reshape(B, N, L, -1) # [B, N, L, d]
        v = self.V(msa).reshape(B, N, L, -1)

        attn = torch.einsum("bihd, bkid->bihk", q, v) * 1/math.sqrt(self.dim)
        attn = nn.functional.softmax(attn, dim=-1)
        gate = torch.sigmoid(self.G(msa)) # [B, L, h, N]

        out = torch.einsum("bihk,bkid->bihd", attn, v).reshape(B, 1, L, -1)
        out *= gate
        out = self.out(out)

        return out
        
class MSAColAttention_mannual(nn.Module):
    def __init__(self,d_msa=64, 
                        n_head=8, 
                        d_hidden=8):
        super().__init__()
        self.Q = nn.Linear(d_msa, n_head*d_hidden, bias=False)
        self.K = nn.Linear(d_msa, n_head*d_hidden, bias=False)
        self.V = nn.Linear(d_msa, n_head*d_hidden, bias=False)

        self.B = nn.Linear(d_pair, n_head, bias=False)
        self.G = nn.Linear(d_msa, n_head*d_hidden)
        self.out = nn.Linear(n_head*d_hidden, d_msa)

        self.h, self.dim = n_head, d_hidden
        #reset parameters
        
    def forward(self, msa):
        # msa: [B, N, L, d_msa]
        B, N, L = msa.shape[:3]
        q = self.Q(msa).reshape(B, N, L, self.h, -1) # [B, N, L, h, d]
        k = self.K(msa).reshape(B, N, L, self.h, -1) 
        v = self.V(msa).reshape(B, N, L, self.h, -1) 

        attn = torch.einsum("bqihd, bkihd->bihqk", q, v) * 1/math.sqrt(self.d_hidden)
        attn = nn.functional.softmax(attn, dim=-1)
        gate = torch.sigmoid(self.G(msa)) # [B, N, L, h, d]

        out = torch.einsum("bihqk,bkihd->bqihd", attn, v).reshape(B, N, L, -1)
        out *= gate
        out = self.out(out)

        return out

* In this **GlobalAttention** mechanism, each query—indexed by batch, position, and head as [B,L,h] —is attended to by all N sequences in the MSA. Specifically, the query at position i interacts with keys from all sequences at the same position, producing attention scores of shape [B,L,h,N] . These scores are then used to compute a weighted sum over the corresponding values [B,N,L,h,d] , aggregating information across the N sequences for each position and head. The result is a context-enhanced representation of shape [B,L,h,d] , which is expanded to [B,1,L,h⋅d] and broadcast only to the first row of the MSA (i.e., the target sequence)
* In line 53 below. This ensures that the global MSA context is injected solely into the primary sequence, while other sequences remain unchanged.

In [286]:
# MSAPairStr2MSA 
# state -> msa -> row_attn -> col_attn -> msa
from rfantibody.rfdiffusion.Attention_module import FeedForwardLayer
from rfantibody.rfdiffusion.util_module import Dropout

class MSAPairStr2MSA_mannual(nn.Module):
    def __init__(self,
                 d_msa=64, 
                 d_pair=128, 
                 n_head=8, 
                 d_state=64,
                 d_hidden=32, 
                 p_drop=0.15, 
                 use_global_attn=True):
        super().__init__()
        self.norm_pair = nn.LayerNorm(d_pair)
        self.proj_pair = nn.Linear(d_pair+36, d_pair)
        self.norm_state = nn.LayerNorm(d_state)
        self.proj_state = nn.Linear(d_state, d_msa)

        self.dropout_row = Dropout(broadcast_dim=1, p_drop=p_drop)

        self.row_attn = MSARowAttentionWithBias_mannual(d_msa=d_msa, 
                                                d_pair=d_pair, 
                                                n_head=n_head, 
                                                d_hidden=d_hidden)
        if use_global_attn:
            self.col_attn = MSAColGlobalAttention_mannual(d_msa=d_msa, n_head=n_head, d_hidden=d_hidden) 
        else:
            self.col_attn = MSAColAttention_mannual(d_msa=d_msa, n_head=n_head, d_hidden=d_hidden) 
        self.ff = FeedForwardLayer(d_msa, 4, p_drop=p_drop)

        # self.reset_paramter()
    def forward(self, msa, pair, rbf_feat, state):
        '''
        Inputs:
            - msa: MSA feature (B, N, L, d_msa)
            - pair: Pair feature (B, L, L, d_pair)
            - rbf_feat: Ca-Ca distance feature calculated from xyz coordinates (B, L, L, 36)
            - xyz: xyz coordinates (B, L, n_atom, 3)
            - state: updated node features after SE(3)-Transformer layer (B, L, d_state)
        Output:
            - msa: Updated MSA feature (B, N, L, d_msa)
        '''

        B, N, L = msa.shape[:3]
        pair = self.norm_pair(pair)
        pair = torch.cat([pair, rbf_feat], axis=-1) # [B, L, L, d_pair+36]
        pair = self.proj_pair(pair) 

        state = self.norm_state(state)
        state = self.proj_state(state).unsqueeze(1)

        msa.index_add_(1, torch.tensor([0,], device=state.device), state) # only add state to the very first row of MSA
        msa += self.dropout_row(self.row_attn(msa, pair))
        msa += self.col_attn(msa)
        msa += self.ff(msa)

        return msa

In [288]:
msa2msa_m = MSAPairStr2MSA_mannual(d_msa=d_msa_full, 
                         d_pair=d_pair,
                          n_head=n_head_msa,
                          d_state=64, #SE3_param['l0_out_features'],
                          use_global_attn=True,
                          d_hidden=32, 
                          p_drop=p_drop).to(device)


msa_m = msa2msa_m(msa_full_ot, 
        pair, 
        rbf_feat, 
        state_update) # [1, 1, 10, 64]

In [300]:
# Then followed by MSA2Pair Module

from rfantibody.rfdiffusion.Track_module import MSA2Pair

msa2pair = MSA2Pair(d_msa=d_msa_full, d_pair=d_pair,
                                 d_hidden=d_hidden//2, p_drop=p_drop).to(device)

pair = msa2pair(msa_, pair) # [1, 10, 10, 128]

* Let's explain how MSA2Pair module work!
* The input for this module are pair information and msa infos, we firstly transform the msa to match the shape of pair then add to pair.

In [312]:

class MSA2Pair_mannual(nn.Module):
    def __init__(self, d_msa=64, 
                         d_pair=128,
                         d_hidden=d_hidden//2, 
                         p_drop=p_drop):
        super().__init__()
        
        self.msa_norm = nn.LayerNorm(d_msa)
        self.msa_left = nn.Linear(d_msa, d_hidden)
        self.msa_right = nn.Linear(d_msa, d_hidden)

        self.out_proj = nn.Linear(d_hidden*d_hidden, d_pair)

        # reset parameters
    def forward(self, msa, pair):
        """
        msa: [B, N, L, d_msa]
        pair: [B, L, L, d_pair]
        """
        B, N, L = msa.shape[:3]

        msa = self.msa_norm(msa)
        left = self.msa_left(msa) # [B, N, L, d_hidden]
        right = self.msa_right(msa) / float(N)
        #print(left.shape)
        out = torch.einsum("bnik,bnjl->bijkl", left, right).reshape(B, L, L, -1) # [B, L, L, d*d]
        pair += self.out_proj(out)

        return pair

In [313]:
msa2pair_m = MSA2Pair_mannual(d_msa=d_msa_full, d_pair=d_pair,
                                 d_hidden=d_hidden//2, p_drop=p_drop).to(device)

msa2pair_m(msa_, pair).shape

torch.Size([1, 10, 10, 128])

* After updating msa and pair, then BiasedAxialAttention is inplemented twice for row and column axis to update the pair infos.
* The BiasedAxialAttention is inplemented as above

In [317]:
from rfantibody.rfdiffusion.Track_module import PairStr2Pair

pair2pair = PairStr2Pair(d_pair=d_pair, n_head=n_head_pair, 
                                      d_hidden=d_hidden, p_drop=p_drop).to(device)

pair = pair2pair(pair, rbf_feat)
pair.shape

torch.Size([1, 10, 10, 128])

* Then comes to the key component -- **Str2Str**, which outputs the rotation matrix **R** and translation vector **T**. Then one can update the noised coordinates by $R*x + T$.
* SE(3)-transformer is applied to update the crds here: we need to construct a graph based on Ca crds, and pass it to se3 layer.

In [332]:
from rfantibody.rfdiffusion.Track_module import Str2Str

#SE3_param={'l0_in_features':32, 
#           'l0_out_features':16, 
#           'num_edge_features':32}

str2str = Str2Str(d_msa=d_msa_full, 
                  d_pair=d_pair,
                   d_state=8,
                   SE3_param=SE3_param_full,
                   p_drop=p_drop).to(device)

R_in = torch.eye(3, device=xyz.device).reshape(1,1,3,3).expand(1, 10, -1, -1)
T_in = xyz[:,:,1].clone()

R, T, state, alpha = str2str(msa_, pair, R_in, T_in, xyz, state, idx_pdb[:, :10], motif_mask=motif_mask, top_k=0) 
R.shape, T.shape, state.shape, alpha.shape     

(torch.Size([1, 10, 3, 3]),
 torch.Size([1, 10, 3]),
 torch.Size([1, 10, 8]),
 torch.Size([1, 10, 10, 2]))

In [352]:
from rfantibody.rfdiffusion.Track_module import SCPred
from rfantibody.rfdiffusion.SE3_network import SE3TransformerWrapper
from rfantibody.rfdiffusion.util_module import rbf, make_full_graph, make_topk_graph

class Str2Str_mannual(nn.Module):
    def __init__(self, 
                  d_msa=64, 
                  d_pair=128,
                   d_state=8,
                   SE3_param={'l0_in_features':32, 'l0_out_features':16, 'num_edge_features':32},
                   p_drop=.1):
        super().__init__()

        self.msa_norm = nn.LayerNorm(d_msa)
        self.pair_norm = nn.LayerNorm(d_pair)
        self.state_norm = nn.LayerNorm(d_state)

        self.embed_x = nn.Linear(d_msa+d_state, SE3_param['l0_in_features'])
        self.embed_e1 = nn.Linear(d_pair, SE3_param['num_edge_features'])
        self.embed_e2 = nn.Linear(SE3_param['num_edge_features']+36+1, SE3_param['num_edge_features'])
        
        self.norm_node = nn.LayerNorm(SE3_param['l0_in_features'])
        self.norm_edge1 = nn.LayerNorm(SE3_param['num_edge_features'])
        self.norm_edge2 = nn.LayerNorm(SE3_param['num_edge_features'])

        self.se3 = SE3TransformerWrapper(**SE3_param)
        self.sc_predictor = SCPred(d_msa=d_msa, d_state=SE3_param['l0_out_features'],
                                   p_drop=p_drop)

        # reset parameters

    def get_seqsep(self, idx):
        pass
        
    def forward(self, msa, pair, R_in, T_in, xyz, state, idx, motif_mask=motif_mask, top_k=0):
        B, N, L = msa.shape[:3]

        node = self.msa_norm(msa[:, 0, ...]) # norm the 1st seq itself
        pair = self.pair_norm(pair) # [B, L, L, d_pair]
        state = self.state_norm(state)

        node = torch.cat([node, state], axis=-1) # [B, L, d_msa+d_state]
        node = self.norm_node(self.embed_x(node)) # (B, L, x)
        pair = self.norm_edge1(self.embed_e1(pair)) # (B, L, L, x)

        neighbour = self.get_seqsep(idx) # actualy I didnot get what the authors wanna do here!
        rbf_feat = rbf(torch.dist(xyz[:, :, 1, :], xyz[..., 1, :])) # Ca dist [B, L, L, 36]
        pair = self.norm_edge2(
                                self.embed_e2(
                                    torch.cat([pair, rbf_feat, neighbour], axis=-1)
                                            )
                                )

        if top_k == 0: # full graph
            G, edge_feats = make_full_graph(xyz[..., 1, :], apir, idx, top_k=top_k)
        else: # top-k nearest neighbours
            G, edge_feats = make_topk_graph(xyz[..., 1, :], apir, idx, top_k=top_k)
            
        
        l1_feats = xyz - xyz[..., 1, :].unsqueeze(2) # abs dist from Ca
        l1_feats = l1_feats.reshape(B*L, -1, 3) # [B*L, 3, 3]

        shift = self.se3(G, node.reshape(B*L, -1, 1), l1_feats, edge_feats) # a dict with keys ['0', '1']
        state = shift['0'].reshape(B, L, -1) 
        offset = shift['1'].reshape(B, L, 2, 3)
        offset[:, motif_mask, ...] = 0

        delTi = offset[..., 0, :]/10.
        R = offset[..., 1, :]/100.
        Qnorm = torch.sqrt(1 + torch.sum(R*R, dim=-1)) # quaternion normalization factor
        qA, qB, qC, qD = 1/Qnorm, R[:,:,0]/Qnorm, R[:,:,1]/Qnorm, R[:,:,2]/Qnorm # unit quaternion (qA, [qB, qC, qD])

        delRi = torch.zeros([B, L, 3, 3], device=xyz.device)
        """
        exp_R = [
                [a**2+b**2-c**2-d**2, 2*b*c-2*a*d, 2*b*d+2*a*c],
                [2*b*c+2*a*d, a**2-b**2+c**2-d**2, 2*c*d-2*a*b],
                [2*b*d-2*a*c, 2*c*d+2*a*b, a**2-b**2-c**2+d**2]
                ]
        """
        delRi[..., 0, 0] = qA**2 + qB**2 - qC**2 - qD**2
        delRi[..., 0, 1] = 2*qB*qC
        delRi[..., 0, 2] = 2*qB*qD + 2*qA*qC
        delRi[..., 1, 0] = 2*qB*qC + 2*qA*qD
        delRi[..., 1, 1] = qA**2 - qB**2 + qC**2-qD**2
        delRi[..., 1, 2] = 2*qC*qD - 2*qA*qB
        delRi[..., 2, 0] = 2*qB*qD - 2*qA*qC
        delRi[..., 2, 1] = 2*qC*qD + 2*qA*qB
        delRi[..., 2, 2] = qA**2 - qB**2 - qC**2 + qD**2

        Ri = torch.einsum("bnij, bnjk->bnik", delRi, R_in)
        Ti = delTi + T_in

        alpha = self.sc_predictor(msa[:, 0], state)

        return Ri, Ti, state, alpha

In [353]:
str2str_m = Str2Str_mannual(d_msa=d_msa_full, 
                  d_pair=d_pair,
                   d_state=8,
                   SE3_param=SE3_param_full,
                   p_drop=p_drop).to(device)


R_m, T_m, state_m, alpha_m = str2str(msa_, pair, R_in, T_in, xyz, state, idx_pdb[:, :10], motif_mask=motif_mask, top_k=0)

* In the above code, the authors use quaternion to construct rotation matrix. For a primer on quaternions and their role in 3D rotations, refer to  **[here](https://github.com/kilianmandon/alphafold-decoded/blob/main/tutorials/geometry/geometry.ipynb)** to learn the basics of quaternion and how it works.
* Within the SE(3)-Transformer framework, unit quaternions are generated to represent rotations, ensuring that both the SE(3) transformations and the quaternion-based rotations are equivariant—a key property for maintaining geometric consistency in 3D deep learning.